# CODE

In [41]:
import panel as pn
import hvplot.pandas
import pandas as pd
import numpy as np
import datetime as dt
import holoviews as hv
import requests

def main():
    # Constants
    # =================================================
    csv_path = r'visualizer_imput.csv'
    count_color = 'blue'
    mean_color = 'red'
    ci_color = 'pink'
    keyword = 'hiv'
    start_date = '2020/01/01'
    end_date = '2020/08/01'
    logo_path = r'https://images.pexels.com/photos/3109167/pexels-photo-3109167.jpeg'
    return visualizer(csv_path, logo_path, keyword, start_date, end_date, count_color, mean_color, ci_color)

def format_df_from_csv(csv_path, sep_str = ',', header_row = 0):

    t_df = pd.read_csv(csv_path, 
                    sep = sep_str, 
                    header = header_row)

    # Replace all empty strings with NA
    t_df.replace('', np.nan, inplace = True)

    # Format Keywords as List (from string version of set)
    # =========================================================
    t_df['Keywords'].replace(np.nan, '[]', inplace = True)
    t_df['Keywords'] = t_df['Keywords'].str.replace('{', "[", regex = True)
    t_df['Keywords'] = t_df['Keywords'].str.replace('}', "]", regex = True)

    # Transform so each row is a unique combination of PMID and author:
    # =========================================================
    t_df['Authors'] = t_df['Authors'].apply(eval)
    t_df = t_df.explode(['Authors']).reset_index(drop = True)
    t_df = t_df.join(pd.json_normalize(t_df.pop('Authors')))
    t_df.replace('', np.nan, inplace = True)

    # Calculate the Number of Authors for each PMID:
    # =========================================================
    t_df['Num_Authors'] = t_df.groupby('Last')['Last'].transform('count')
    # Convert count for articles without a listed author to 0, make val an integer
    t_df.loc[t_df['Last'].isna(), 'Num_Authors'] = 0
    t_df['Num_Authors'] = t_df['Num_Authors'].astype(int)
    t_df['Author'] = t_df['Last'] + ', ' + t_df['Initials']
    t_df.loc[t_df['Last'].isna(), 'Author'] == 'None Listed'
    # For the rare authors that are only stored in last, typically groups
    t_df['Author'].fillna(t_df['Last'], inplace=True)

    # Convert Order into a True/False, for First Author
    # =========================================================
    t_df.loc[t_df['Order'] != '1', 'Order'] = False
    t_df.loc[t_df['Order'] == '1', 'Order'] = True
    t_df = t_df.rename(columns = {'Order': 'FirstAuthor'})

    # Convert language values in 'Language' to full names
    # =========================================================
    # Uses the Library of Congress Language Abbreviations, which
    # is what is used by PubMed, to create a list translating
    # each 3-letter abbreviation into the corresponding language
    # in full text
    dict_path = r'https://www.loc.gov/marc/languages/language_code.html'
    r = requests.get(dict_path)
    lang_dict = pd.read_html(r.text)[0]
    lang_dict['code'] = "'" + lang_dict['code'] + "'"
    lang_dict['language'] = "'" + lang_dict['language'] + "'"
    lang_dict = dict(zip(lang_dict['code'], lang_dict['language']))

    # Ensure all languages have single quotes, not double quotes:
    t_df['Language'] = t_df['Language'].str.replace('"', "'", regex = True)
    # Replace text using regular expressions
    for key in lang_dict.keys():
        t_df['Language'] = t_df['Language'].str.replace(key, lang_dict[key], regex = True)
        
    # Create Month Interval Values
    # =========================================================
    t_df['PubDate'] = pd.to_datetime(t_df['PubDate'], format =r'%Y-%m-%d')
    # Round down to the start of the month, regardless of day value
    t_df['PubYM'] = t_df['PubDate'].dt.to_period('M').dt.to_timestamp()
    t_df['PubYM'] = t_df['PubYM'].astype(str)

    month_ints = pd.date_range(start = min(t_df['PubDate']), 
                end = max(t_df['PubDate']), 
                freq = 'MS'
                ).strftime("%Y-%m-01").tolist()
    int_key = {}
    for i_interval in month_ints:
        int_key[i_interval] = month_ints.index(i_interval) + 1

    t_df['MonthInterval'] = t_df['PubYM']
    t_df['MonthInterval'] = t_df['MonthInterval'].map(int_key)

    return t_df

def filter_transform_df(
    df,
    dates,
    journal,
    num_authors,
    languages
    ):
    subset = df
    # Subset based on criteria
    # ==================================================
    # Check after each step to ensure the subset is not
    # empty, which would throw an error if subsetting
    # was allowed to proceed. If it is empty, stop
    # subsetting, and move to processing.
    while True:
        # Dates 
        subset = subset[subset['PubDate'].between(dates[0], dates[1])]
        if subset.shape[0] == 0:
            break
        # Journal
        subset = subset[subset['Journal'].isin(journal)]
        if subset.shape[0] == 0:
            break       
        # Number of Authors
        subset = subset[subset['Num_Authors'].between(num_authors[0], num_authors[1])]
        if subset.shape[0] == 0:
            break   
        # Languages
        subset = subset[subset['Language'].str.contains('|'.join(languages))]
        if subset.shape[0] == 0:
            break   
        # if fully reached the end of subsetting, BREAK!
        break

    # Convert to Date - Count Format
    # ==================================================
    # Pivot the table to a list of dates within the range
    # of the query, with counts of articles for each month.
    # If the subset is empty, return counts of zero. 
    # zero, with the subset table and full table
    # equal - this avoids errors with the visualizer
    # by avoiding an empty input.

    dates = pd.date_range(start = dates[0], end = dates[1],
            freq = 'MS'
            ).tolist()
    dates = pd.to_datetime(dates, format =r'%Y-%m-%d').rename('PubDate')

    counts_table = pd.DataFrame(dates)

    if subset.shape[0] == 0:
        counts_table['NumCounts'] = 0

    base_counts = subset[subset['FirstAuthor']].groupby('PubYM')['PubYM'].count().T.to_dict()
    counts_table['NumCounts'] = counts_table['PubDate'].astype(str).replace(base_counts)
    counts_table['NumCounts'] = pd.to_numeric(counts_table['NumCounts'], errors='coerce').fillna(0, downcast='int') 
    counts_table['Statistic Value'] = counts_table['NumCounts']    
    return counts_table

def create_line_plot(data_frame, count_color, mean_color, ci_color):

    line_plot = data_frame.hvplot.line(
        x='PubDate', 
        y='NumCounts', 
        title = 'Articles per Month', 
        xlabel = 'Published Date', ylabel = 'Number of Articles', 
        line_color = count_color,
        legend='top', height=250, width=500)

    mean_line = hv.HLine(data_frame['NumCounts'].describe()['mean'])
    mean_line.opts(
        color=mean_color, 
        line_dash='dashed', 
        line_width=1.0,
    )

    highci_line = hv.HLine(max(data_frame['NumCounts'].describe()['mean'] - 1.96 * data_frame['NumCounts'].describe()['std'],0))
    highci_line.opts(
        color=ci_color,  
        line_width=1.0,
    )

    lowci_line = hv.HLine(min(data_frame['NumCounts'].describe()['mean'] + 1.96 * data_frame['NumCounts'].describe()['std'], max(data_frame['NumCounts'])))
    lowci_line.opts(
        color=ci_color, 
        line_width=1.0,
    )

    plot = line_plot * mean_line * highci_line * lowci_line
    return plot

def subset_dates(counts_table):
    if counts_table['NumCounts'].max() == 0:
        out_table = counts_table.loc[0]
    else:
        max_date = counts_table[counts_table['NumCounts'] != 0]['PubDate'].max()
        min_date = counts_table[counts_table['NumCounts'] != 0]['PubDate'].min()
        out_table = counts_table[counts_table['PubDate'].between(min_date, max_date)]
    return out_table

def describe_stats(table):
    table['Statistic Value'] = table['NumCounts']
    return table['Statistic Value'].describe().rename(index = {'count': 'months'})

def date_range_text(counts_table):
   start_date = counts_table['PubDate'].min().strftime(format = '%B %Y')
   end_date = counts_table['PubDate'].max().strftime(format = '%B %Y')
   count = counts_table['NumCounts'].sum()
   return f'### Articles between {start_date} --  {end_date}: {count:,} articles'




def visualizer(csv_path, logo_path, keyword, start_date, end_date, count_color, mean_color, ci_color):

    # Constants
    # =================================================
    # Extract and Format DataFrame from CSV
    # -------------------------------------------------

    vis_df = format_df_from_csv(csv_path, sep_str = ',', header_row = 0)

    # Constants from Data Frame
    # -------------------------------------------------
    min_date = vis_df['PubDate'].min()
    min_date = dt.datetime(min_date.year, min_date.month, 1)
    max_date = vis_df['PubDate'].max()
    max_date = dt.datetime(max_date.year, max_date.month, 1)

    # Format Logo and Subtitle
    # -------------------------------------------------
    logo = pn.panel(logo_path, width=200, align='start')
    text = ''.join(['Dataset: PubMed records for articles published between ',
        f'{pd.to_datetime(start_date, format = r"%Y/%m/%d").strftime(format = r"%B %d, %Y")} and ',
        f'{pd.to_datetime(end_date, format = r"%Y/%m/%d").strftime(format = r"%B %d, %Y")} ',
        f' for search term: {keyword}'
        ])

    # Title 
    # -------------------------------------------------
    title = '# Articles per Month'

    # Build Visualizer Widgets
    # =================================================

    # Date Range Selection Widget
    # -------------------------------------------------
    date_widget = pn.widgets.DateRangeSlider(
        name = 'Date Range',
        start = min_date, 
        end = max_date,
        value = (min_date, max_date),
        step = 1,
        value_throttled = (min_date, max_date)
    )

    # Journal Selection Widget
    # -------------------------------------------------
    journals_list = sorted(vis_df.Journal.unique().tolist())
    journal_widget = pn.widgets.MultiSelect(
    name = "Journal Name", 
    value = journals_list, 
    options = journals_list
    )

    # Number of Authors Widget
    # -------------------------------------------------
    longest_authors = int(vis_df['Num_Authors'].max())
    author_range_widget = pn.widgets.RangeSlider(
        name = 'Number of Authors', 
        start = 0, 
        end = longest_authors, 
        value = (0, longest_authors), 
        step = 1,
        value_throttled = (0, longest_authors)
        )

    # Language Widget
    # -------------------------------------------------
    languages_list = vis_df['Language'].apply(eval).explode('Language').unique().tolist()
    language_widget = pn.widgets.MultiSelect(
    name = "Languages", 
    value = languages_list, 
    options = languages_list
    )

    # Functions that are bound to interactable parameters
    # =====================================================
    @pn.depends(date_widget.param.value_throttled, journal_widget.param.value, author_range_widget.param.value, language_widget.param.value)
    def call_bulk_df(
        dates = date_widget.value_throttled,
        journal = journal_widget.value,
        num_authors = author_range_widget.value,
        languages = language_widget.value
        ):
        return filter_transform_df(vis_df, dates, journal, num_authors, languages)
        
    bulk_text = pn.bind(date_range_text, counts_table = call_bulk_df)
    bulk_stats = pn.bind(describe_stats, table = call_bulk_df)
    bulk_plot = pn.bind(create_line_plot, data_frame = call_bulk_df, 
                        count_color = count_color, 
                        mean_color = mean_color, 
                        ci_color = ci_color)
    subset_data = pn.bind(subset_dates, counts_table = call_bulk_df)
    subset_text = pn.bind(date_range_text, counts_table = subset_data)
    subset_stats = pn.bind(describe_stats, table = subset_data)
    subset_plot = pn.bind(create_line_plot, data_frame = subset_data, 
                        count_color = count_color, 
                        mean_color = mean_color, 
                        ci_color = ci_color)

    # VISUALIZER
    # =======================================================

    gspec = pn.GridSpec(sizing_mode='stretch_both', max_height=1200)

    # Header Panel
    gspec[0,   0:] = pn.Row(
        logo, 
        pn.Column(pn.pane.Markdown(title), pn.pane.Markdown("#### " + text),
                pn.pane.Markdown('Royalty free image from pexels')),
        height_policy = 'min', 
        height = 200,
        align="center"
        )
    # Sidebar Panel - Widgets
    gspec[1:,   0] = pn.WidgetBox(
        pn.pane.Markdown("### Subset Selectors", align = 'center'),
        pn.layout.Divider(),
        date_widget, 
        author_range_widget, 
        language_widget, 
        journal_widget,
        width = 200,
        align = "center",
        width_policy = 'min'
        )
    # Summary Stats and Time Plot for total range
    gspec[1:3,   1:4] = pn.Card(
        bulk_text,
        pn.Row(pn.Column(bulk_stats), 
            pn.WidgetBox(bulk_plot, height_policy ='min', sizing_mode = 'scale_both')
            ),
        pn.layout.Divider()
        )
    # Summary Stats and Time Plot for range with articles
    gspec[3:6,   1:4] = pn.Card(
        subset_text, 
        pn.pane.Markdown('Subset to exclude leading or trailing months without articles.'),
        pn.Row(pn.Column(subset_stats), 
            pn.WidgetBox(subset_plot, height_policy ='min', sizing_mode = 'scale_both')
            ))
    
    return gspec

In [ ]:
'''
THIS WOULD BE USED FOR SCRIP FUNCTIONALITY, BUT NOT THE JUPYTER NOTEBOOK - MAKES A DEPLOYABLE WEB APP!
if __name__.startswith("bokeh"):
    # start with panel serve script.py
    dashboard = visualizer
    dashboard.servable()
if __name__ == "__main__":
    # start with python script.py
    dashboard = visualizer
    dashboard.show(port=5007)
    '''

This is used for the jupyter notebook. It may look like it generates like, 4 empty outputs. That's normal

In [ ]:
pn.extension()

# Output

In [43]:
main()

BokehModel(combine_events=True, render_bundle={'docs_json': {'0e6dcd08-5f19-427d-91e5-0c773a541281': {'version…